In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 ***1. Training the data***

In [ ]:
data=pd.read_csv('/kaggle/input/titanic/train.csv')
data

In [ ]:
#sns.heatmap(data = data,annot=True)

 ***2. Replace the values in 'Embarked' column and make separate values for each of the following values it contains in that column. 
Note :  We are converting strings into int to prevent further errors that would occur in the followig steps***

In [ ]:
data.Embarked.unique()
data['Embarked_s']=data['Embarked'].replace('Q',0).replace('C',0).replace('S',1)
data['Embarked_c']=data['Embarked'].replace('S',0).replace('Q',0).replace('C',1)
data['Embarked_q']=data['Embarked'].replace('C',0).replace('S',0).replace('Q',1)

***Note: Conversion of str into int to avoid errors. ***

In [ ]:
#data.Sex= data.Sex.map( {'M': 1, 'F': 0, 'O': 3} )
data['Sex'].replace( 'female', 0, inplace=True )
data['Sex'].replace( 'male', 1, inplace=True )
data

***If you want to remove 0 value in 'Embarked' column then simply run the code given below and get the required data .***


In [ ]:
#data=data[data.Embarked!=0]
#data=data[data[('Embarked')].map(data['Embarked'].value_counts())>0]*
#data

In [ ]:
data.isnull().sum()

***Inorder to fill the null values , simply type the follwing code given below.
Note:There is an alterative method to remove null values i.e 'IMPUTER . Check out 'trans'(will be done in the folllwing steps) where we simply impute the null values based on parameters applied to a specific imputer.*** 

In [ ]:
import seaborn as sns
sns.countplot(data.Survived)

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
import missingno
missingno.matrix(data)

In [ ]:
sns.factorplot('Survived','Age',data=data,size=5)

In [ ]:
import seaborn as sns
sns.barplot(x="Sex", y="Survived", data=data, palette="hls")

In [ ]:
import matplotlib.pyplot as plt
se = sns.factorplot(x = 'Pclass', y = 'Survived', hue = 'Sex', data = data, kind = 'bar',palette='rocket')
se.despine(left = True)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Sex and Passenger Class')

sw = sns.factorplot(x = 'SibSp', y = 'Survived', hue = 'Sex', data = data, kind = 'bar',palette='Set2',size=7.5)
se.despine(left = False)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Sex and SibSp')

ss = sns.factorplot(x = 'Embarked', y = 'Survived', hue = 'Sex', data = data, kind = 'bar',palette="hls")      #palette=mako , flare, crest , magma
se.despine(left = False)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Sex and SibSp')

In [ ]:
sns.countplot(x='SibSp',data=data,palette="mako")

In [ ]:
data['Age'].hist(bins=25,color='darkred',alpha=0.6)
sns.factorplot(x="Survived", y ="Embarked", data=data, kind="bar", size=4, palette="cubehelix")
plt.show()

In [ ]:
sns.set(style='whitegrid')
sns.boxplot(x=data['Age'],linewidth=3.5,palette="crest")

In [ ]:
sns.factorplot(x = "Sex", y = "Age", hue = "Pclass",data = data, kind = "box",palette="mako",size=6.6)
plt.show()

In [ ]:
y = data.Age.values.reshape(-1,1)
x = data.Fare.values.reshape(-1,1)

plt.scatter(x,y,color='orange')
plt.ylabel("Age")
plt.xlabel("Fare")

In [ ]:
pd.crosstab([data.Embarked],[data.Age,data.Survived],margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
sns.factorplot('Sex','Pclass',data=data)
plt.show()

In [ ]:
#data.Sex=data.Sex.fillna(0)
#data.Age=data.Age.fillna(0)
#data.Cabin=data.Cabin.fillna(0)
#data.Embarked_s=data.Embarked_s.fillna(0)
#data.Embarked_c=data.Embarked_c.fillna(0)
#data.Embarked_q=data.Embarked_q.fillna(0)


In [ ]:
data['Survived']

***Since we have not removed the null values we impute the null values*** 

In [ ]:
data.isnull().sum()

In [ ]:
data.dtypes

***3. We first drop out the "survived" column from the data inorder to find its prediction by using varous algorithms. And then remove the unecessary columns that isnt really necesaary in further steps .***

In [ ]:
x = data.drop(columns = ['Survived','Ticket','Cabin','PassengerId','Name','Embarked'])

In [ ]:
y = data[['Survived']]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)

In [ ]:
x.head()

In [ ]:
x.dtypes

In [ ]:
x

In [ ]:
x.Embarked_q.unique()

In [ ]:
nom_cols =[]
ord_cols = []
num_cols=[0,4,5]
null_cols=[1,2,6,7,8]
#Kbin_cols=[0,1,4],
binarizer_cols=[3]

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,KBinsDiscretizer,Binarizer
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.compose import make_column_transformer
from sklearn import set_config
from sklearn.preprocessing import StandardScaler

trans = make_column_transformer((OneHotEncoder(sparse=False),nom_cols),
                                (KNNImputer(n_neighbors=45),null_cols),
                                (OrdinalEncoder(), ord_cols),(StandardScaler(),num_cols)
                                 ,(Binarizer(threshold=55),binarizer_cols),remainder= 'passthrough')
set_config(display= 'diagram')
trans
#trans = make_column_transformer((OneHotEncoder(sparse=False),nom_cols),(SimpleImputer(strategy='mean'),null_cols),
                                 #(OrdinalEncoder(), ord_cols),(StandardScaler(),num_cols),(KBinsDiscretizer(),Kbins_cols)
                                 #,(Binarizer(threshold=55),binarizer_cols),remainder= 'passthrough')
#set_config(display= 'diagram')
#trans


In [ ]:
from sklearn.tree import DecisionTreeClassifier 
algorithm_x=DecisionTreeClassifier(criterion='entropy')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
algorithm = KNeighborsClassifier(15) #
algorithm

In [ ]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(trans,algorithm)
pipe_x=make_pipeline(trans,algorithm_x)
pipe


In [ ]:
pipe_x

In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
pipe_x.fit(x_train,y_train)


In [ ]:
pred_x=pipe_x.predict(x_test)
pred_x

In [ ]:
from sklearn.metrics import accuracy_score, plot_confusion_matrix
accuracy_score(pred_x,y_test)*100

In [ ]:
pred=pipe.predict(x_test)
pred

In [ ]:
from sklearn.metrics import accuracy_score, plot_confusion_matrix
accuracy_score(pred,y_test)*100

In [ ]:
import matplotlib as plt 
plot_confusion_matrix(pipe,x_test,y_test)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
over= RandomOverSampler()
o_x , o_y = over.fit_resample(x,y)
o_y.value_counts()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc=GradientBoostingClassifier()
pipe_gbc=make_pipeline(trans,gbc)
pipe_gbc

In [ ]:
pipe_gbc.fit(x_train,y_train)

In [ ]:
pred_gbc=pipe_gbc.predict(x_test)
pred_gbc

In [ ]:
accuracy_score(pred_gbc,y_test)*100

In [ ]:
from imblearn.over_sampling import SMOTE
over= RandomOverSampler()
_x , o_y = over.fit_resample(x,y)
o_y.value_counts()

In [ ]:
from imblearn.pipeline import make_pipeline
s = SMOTE()
n_pipe = make_pipeline(trans,s,gbc)
n_pipe

In [ ]:
accuracy_score(pred,y_test)*100


In [ ]:
from sklearn.linear_model import LogisticRegression as lr
algorithm_2 = lr(solver='liblinear')
pipe_2 = make_pipeline(trans,algorithm_2)
pipe_2

In [ ]:
pipe_2.fit(x_train,y_train)


In [ ]:
pred_2 = pipe_2.predict(x_test)
pred_2

In [ ]:
accuracy_score(pred_2,y_test)*100

In [ ]:
from sklearn.model_selection import KFold, cross_val_score,StratifiedKFold
kf=StratifiedKFold(n_splits=4)
np.mean(cross_val_score(pipe,x,y,cv=kf,scoring='accuracy')*100) 

In [ ]:
from sklearn.svm import SVC
model1=SVC(kernel='linear')
pipe_1 = make_pipeline(trans,algorithm)
pipe_1
pipe_1.fit(x_train,y_train)
pred_1=pipe.predict(x_test)
accuracy_score(pred_1,y_test)*100

In [ ]:
from sklearn.ensemble import BaggingClassifier
model2=BaggingClassifier(base_estimator=SVC(3))
model2

In [ ]:
y_test.shape

In [ ]:
pipe5=make_pipeline(trans,model2)
pipe5

In [ ]:
pipe5.fit(x_train,y_train)
pred5=pipe5.predict(x_test)


In [ ]:
pred5.shape

In [ ]:
accuracy_score(pred5,y_test)*100


In [ ]:
data1=pd.read_csv('/kaggle/input/titanic/test.csv')
data1

In [ ]:
missingno.matrix(data1)

In [ ]:
data1['Sex'].replace( 'female', 0, inplace=True )
data1['Sex'].replace( 'male', 1, inplace=True )

In [ ]:
data1.isnull().sum()

In [ ]:
data1.Age=data1.Age.fillna(0)
data1.Cabin=data1.Cabin.fillna(0)
data1.Fare=data1.Fare.fillna(0)


In [ ]:
data1.isnull().sum()

In [ ]:
#data1=data1[data1.Embarked!=0]
#data1=data1[data1[('Embarked')].map(data1['Embarked'].value_counts())>0]
#data1

In [ ]:
#data1.loc[data1['Sex']=='male']=1
#data1.loc[data1['Sex']=='female']=0

In [ ]:
data1['Embarked_s']=data1['Embarked'].replace('Q',0).replace('C',0).replace('S',1)
data1['Embarked_c']=data1['Embarked'].replace('S',0).replace('Q',0).replace('C',1)
data1['Embarked_q']=data1['Embarked'].replace('C',0).replace('S',0).replace('Q',1)

In [ ]:
data1.head()

In [ ]:
x1 = data1.drop(columns = ['Ticket','Cabin','PassengerId','Name','Embarked'])
x1

In [ ]:
x1.isnull().sum()

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,KBinsDiscretizer,Binarizer
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.compose import make_column_transformer
from sklearn import set_config
from sklearn.preprocessing import StandardScaler

trans = make_column_transformer((OneHotEncoder(sparse=False),nom_cols),
                                (KNNImputer(n_neighbors=45),null_cols),
                                (OrdinalEncoder(), ord_cols),(StandardScaler(),num_cols)
                                 ,(Binarizer(threshold=55),binarizer_cols),remainder= 'passthrough')
set_config(display= 'diagram')
trans

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt
#data1=np.random.randint(low = 1,high = 100,size = (10, 10))
sns.heatmap(data=data1.select_dtypes('int','float'),annot=True)
plt.show()

In [ ]:
pred123=pipe_gbc.predict(x1)
pred123

In [ ]:
#survival_prediction=algorithm.predict(x1)
#survival_prediction

In [ ]:
submit=pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [ ]:
submit['Survived']=pred123
submit.to_csv('submission.csv', index=False)

In [ ]:
submit.Survived.value_counts()